In [1]:
import random
import rich
import sys
import json
from collections import Counter
import os
sys.path.append("/home/wenkail/diplomacy/sotopia-diplomacy/src")
from tqdm import tqdm
from sotopia.database import AgentProfile, EpisodeLog, EnvironmentProfile

In [2]:
from profile_utils import store_env_profile, read_games_from_folder, random_country_adjacency_selection, store_env_profile_with_previous
import argparse
from tqdm import tqdm
import os
import json
import pandas as pd
import pdb
import random

def add_env_profiles(games_dir, games_phases, tag, game_dir):
    for game_phases in tqdm(games_phases, desc="Processing games"):
        # TODO: Get game here
        # pdb.set_trace()
        game = json.load(open(games_dir + game_phases['game_id'] + '.json'))
        for phase in game['phases']:
            if phase['name'] == game_phases['phase']:
                # store_env_profile(game_phases['game_id'], phase, game_phases['countries'], tag)
                store_env_profile_with_previous(game_phases['game_id'], phase, game_phases['countries'], tag, game_dir)

/home/wenkail/miniconda3/envs/sotopia/lib/python3.11/site-packages/beartype/_util/hint/pep/utilpeptest.py:339: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Sequence[sotopia.agents.base_agent.BaseAgent[sotopia.messages.message_classes.Observation, sotopia.messages.message_classes.AgentAction]] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [3]:
folder_path = "/data/user_data/wenkail/sotopia_diplomacy/whole_filter_games_100"
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

games = []
num = 0
for file_path in tqdm(file_paths):
    with open(file_path, 'r') as f:
        games.append(json.load(f))

100%|██████████| 100/100 [00:00<00:00, 486.63it/s]


In [4]:
valid_countries = ['Austria', 'England', 'France', 'Germany', 'Italy', 'Russia', 'Turkey']
upper_countries = [c.upper() for c in valid_countries]

In [5]:
threshold_dict = {}
for threshold in range(2, 12):
    choice_phases_list = []
    for game in games:
        
        for phase in game['phases']:
            choice_phase = {}
            counter = Counter()
            messages = phase['messages']
            counter = Counter()

            for msg in messages:
                pair = tuple(sorted([msg['sender'], msg['recipient']]))
                counter[pair] += 1
            choice_phase['game_id'] = games[0]['id']
            choice_phase['phase_name'] = phase['name']
            for pair, count in counter.items():
                if count >= threshold:
                    choice_phase['countries'] = list(pair)
                    choice_phase['message_count'] = count
                    choice_phases_list.append(choice_phase)
    threshold_dict[threshold] = len(choice_phases_list)

In [6]:
threshold_dict

{2: 3350,
 3: 2506,
 4: 1826,
 5: 1361,
 6: 1011,
 7: 749,
 8: 551,
 9: 425,
 10: 334,
 11: 260}

In [7]:
from collections import Counter

threshold_dict = {}
choice_phases_list = []

for game in games:
    for phase in game['phases']:
        messages = phase['messages']
        counter = Counter()

        for msg in messages:
            pair = tuple(sorted([msg['sender'], msg['recipient']]))
            counter[pair] += 1

        for pair, count in counter.items():
            if count >= 3 and count <= 6:
                choice_phase = {}
                choice_phase['game_id'] = game['id']
                choice_phase['phase'] = phase['name']
                countries = list(pair)
                countries = [c.capitalize() for c in countries]
                choice_phase['countries'] = countries
                choice_phase['message_count'] = count
                choice_phases_list.append(choice_phase)
                # Remove the `break` to consider all pairs meeting the threshold


In [8]:
len(choice_phases_list)

1757

In [9]:
# for i in choice_phases_list:
#     print(i)

In [10]:
sub_choice_phases_list = choice_phases_list
games_dir = "/data/user_data/wenkail/sotopia_diplomacy/whole_filter_games_100/"
tag = "taskeval_fewshot_v3"
add_env_profiles(games_dir, sub_choice_phases_list, tag, games_dir)

Processing games: 100%|██████████| 1757/1757 [00:04<00:00, 387.35it/s]


### Test Games

In [12]:
game_dir = "/data/user_data/wenkail/sotopia_diplomacy/whole_filter_games_100/"
game_id = "27465"
phase_name = "F1906M"
with open(game_dir + game_id + ".json") as f:
    game = json.load(f)

In [13]:
previous_phase = []
for phase in game['phases']:
    if phase['name'] == phase_name:
        print(phase)
    else:
        previous_phase.append(phase)


{'name': 'F1906M', 'state': {'timestamp': 1716556638338561, 'zobrist_hash': '6055585612385193501', 'note': '', 'name': 'F1906M', 'units': {'AUSTRIA': ['A VIE', 'A UKR'], 'ENGLAND': [], 'FRANCE': ['F WES', 'F NTH', 'F TYS', 'A BUR', 'A BEL', 'A PIE', 'F IRI', 'F NWG', 'F MAO'], 'GERMANY': ['F BAL', 'A SIL', 'A MUN', 'A BER', 'F NWY', 'A GAL', 'A FIN'], 'ITALY': ['A TRI', 'F TUN', 'F NAP', 'A TYR', 'A VEN'], 'RUSSIA': ['A MOS', 'A BUD', 'A LVN', 'F PRU', 'A WAR'], 'TURKEY': ['A SER', 'F ION', 'A BUL', 'F NAF', 'A ALB', 'F GRE']}, 'retreats': {'AUSTRIA': {}, 'ENGLAND': {}, 'FRANCE': {}, 'GERMANY': {}, 'ITALY': {}, 'RUSSIA': {}, 'TURKEY': {}}, 'centers': {'AUSTRIA': ['VIE', 'WAR'], 'ENGLAND': [], 'FRANCE': ['BRE', 'MAR', 'PAR', 'BEL', 'POR', 'SPA', 'LVP', 'LON', 'EDI'], 'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL', 'SWE', 'NWY'], 'ITALY': ['NAP', 'TUN', 'ROM', 'TRI', 'VEN'], 'RUSSIA': ['MOS', 'SEV', 'STP', 'RUM', 'BUD'], 'TURKEY': ['ANK', 'CON', 'SMY', 'BUL', 'SER', 'GRE']}, 'homes': {'AU

In [14]:
countries = ['GERMANY', 'FRANCE']
dialogue_unit = ""
for phase in previous_phase:
    dialogue_unit += f"{phase['name']}: \n"
    dialogue_unit += f"Dialogue Between Two Countries: \n"
    for message in phase['messages']:
        if message['sender'] in countries and message['recipient'] in countries:
            clean_message = message['message'].replace("\n", " ")
            dialogue_unit += f"{message['sender']} to {message['recipient']}: {clean_message}\n"

    dialogue_unit += f"Countries' Center in This Phase: \n"
    dialogue_unit += str(phase['state']['centers']) + "\n"
    dialogue_unit += f"Countries' Units in This Phase: \n"
    dialogue_unit += str(phase['state']['units']) + "\n"

In [15]:
str(previous_phase[0]['state']['units'])


"{'AUSTRIA': ['A SER', 'A TYR', 'A GAL', 'A VEN', 'A BUD'], 'ENGLAND': ['F NTH', 'A YOR', 'F NWY', 'F CLY'], 'FRANCE': ['A BEL', 'A PIC', 'F ENG', 'A BUR', 'F NAO', 'F MAO'], 'GERMANY': ['F BAL', 'A HOL', 'F KIE', 'A MUN', 'A BER'], 'ITALY': ['A TUN', 'F ION', 'F GRE', 'A ROM'], 'RUSSIA': ['A SEV', 'F RUM', 'F SWE', 'A UKR', 'F BOT', 'A WAR'], 'TURKEY': ['A BUL', 'F AEG', 'F EAS', 'A SMY']}"

In [16]:
print(dialogue_unit)

S1903M: 
Dialogue Between Two Countries: 
GERMANY to FRANCE: OK - looking more at the alternative you suggest, which sounds OK to me. I can also move Ber to Sil to keep that Russian free at the very least. I'm guessin you will get your fleet to Irish sea this time.
FRANCE to GERMANY: I will move to Irish sea from MAO, you are right.  There is little thing that could interfere with our plan...that is option for Russia to support himself to Baltic sea. I hope he will not do that, and that he will be more defensive in the north (Austrian build in Budapest should divert his attention to south).
GERMANY to FRANCE: Yeah - I did wonder about that - but just hope Russia doesn't consider it.
Countries' Center in This Phase: 
{'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'VEN'], 'ENGLAND': ['EDI', 'LON', 'LVP', 'NWY'], 'FRANCE': ['BRE', 'MAR', 'PAR', 'BEL', 'POR', 'SPA'], 'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'], 'ITALY': ['NAP', 'ROM', 'TUN', 'GRE'], 'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM

In [17]:
for phase in previous_phase:
    print(phase)

{'name': 'S1903M', 'state': {'timestamp': 1716556638177915, 'zobrist_hash': '5801140274428563239', 'note': '', 'name': 'S1903M', 'units': {'AUSTRIA': ['A SER', 'A TYR', 'A GAL', 'A VEN', 'A BUD'], 'ENGLAND': ['F NTH', 'A YOR', 'F NWY', 'F CLY'], 'FRANCE': ['A BEL', 'A PIC', 'F ENG', 'A BUR', 'F NAO', 'F MAO'], 'GERMANY': ['F BAL', 'A HOL', 'F KIE', 'A MUN', 'A BER'], 'ITALY': ['A TUN', 'F ION', 'F GRE', 'A ROM'], 'RUSSIA': ['A SEV', 'F RUM', 'F SWE', 'A UKR', 'F BOT', 'A WAR'], 'TURKEY': ['A BUL', 'F AEG', 'F EAS', 'A SMY']}, 'retreats': {'AUSTRIA': {}, 'ENGLAND': {}, 'FRANCE': {}, 'GERMANY': {}, 'ITALY': {}, 'RUSSIA': {}, 'TURKEY': {}}, 'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'VEN'], 'ENGLAND': ['EDI', 'LON', 'LVP', 'NWY'], 'FRANCE': ['BRE', 'MAR', 'PAR', 'BEL', 'POR', 'SPA'], 'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'], 'ITALY': ['NAP', 'ROM', 'TUN', 'GRE'], 'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'], 'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}, 'homes': {'AUSTRI